# PHIRI Use case A: Vulnerable populations, inequalities and risk factors with direct and indirect impact on COVID-19 outcomes

***
### Jupyter open Notebook produced by **[PHIRI.eu](http://phiri.eu)**. Resources are available at Zenodo **[DOI](https://doi.org/10.5281/zenodo.6377096)**.
***

Research question: Has the COVID19 pandemic changed existing patterns of non-COVID-19 health care utilisation and mortality for vulnerable populations within and between countries? This is based around answering the indirect effects of the pandemic on population health.

Cohort description: Any patients 18 years old or older, with a hospital episode with a main diagnosis of any of the conditions included in the cohort definition (i.e. heart attack, stroke, hip or knee replacements, or severe trauma) from 2019-01-01 until today.

This Jupyter notebook is licensed under the **[MIT License](https://github.com/PderyckeSciensano/PHIRI/blob/main/LICENSE)**. 

## Use Case A on Vulnerable Populations ##

Wide variations in COVID-19 infection and outcomes exist across Europe and within countries. Use Case A explores the indirect impact of the pandemic on health care utilisation in three tracer groups of conditions - heart attack and stroke, hip and knee replacement and severe trauma. By incorporating deprivation measures, further comparisons examining differences across socioeconomic status can be investigated.

### LOCAL ANALYSES ###

This analyses corresponds with the local part of the analysis (country-specific). 
Please, provide feedback on your outputs, both data quality analysis (EDA), HTML report and aggregated output to the Use Case A leaders to complete the overall analyses.


In [ ]:
#update.packages(ask = FALSE, checkBuilt = TRUE)
options(warn=-1)

## First specify the packages of interest
packages = c("tidyverse","readr","ggplot2","dplyr","lubridate","plotly","DT","janitor")

## Now load or install&load all
package.check <- lapply(
  packages,
  FUN = function(x) {
    if (!require(x, character.only = TRUE)) {
      install.packages(x, dependencies = TRUE)
      library(x, character.only = TRUE)
    }
  }
)

warning = function(x, options) {
     paste('\n\n<div class="alert alert-warning">',
           gsub('##', '\n', gsub('^##\ Warning:', '**Warning**', x)),
           '</div>', sep = '\n')
}

calculate_socecon_lvl_ratio <- function(p1) {
  
  if(!("5" %in% colnames(p1)))
  {
    p1$`5` <- NA
  }
  if(!("4" %in% colnames(p1)))
  {
    p1$`4` <- NA
  }
  if(!("3" %in% colnames(p1)))
  {
    p1$`3` <- NA
  }
  if(!("2" %in% colnames(p1)))
  {
    p1$`2` <- NA
  }
  if(!("1" %in% colnames(p1)))
  {
    p1$`1` <- NA
  }
  p1 <- p1 %>% mutate(ratio = `1` /  `5`)
  return(p1)
}

In [ ]:
library(tidyverse)
library(readr)
library(ggplot2)
library(dplyr)
library(lubridate)
library(plotly)
library(DT)
library(janitor)

In [ ]:
age_max <- 105
age_diff <- 5

# Import synthetic data 
df <- read_delim("./UseCase_A/usecaseA_synthetic.csv", delim = "|", escape_double = FALSE, trim_ws = TRUE)

year_from <- 2017
year_to <- year(max(max(df$date_event_heart,na.rm = TRUE),max(df$date_event_stroke,na.rm = TRUE),max(df$date_event,na.rm = TRUE)))

date_from <- as.Date(paste0(year_from,'-01-01'))
date_to <- as.Date(paste0(year_to,'-12-31'))

print(date_from)

In [ ]:
df <- df %>% mutate(age_group = cut(
  pmin(age_nm, age_max-1),
  seq(0, age_max, age_diff),
  right = FALSE,
  labels = c(
    paste(seq(0,age_max-2*age_diff,age_diff), seq(age_diff-1,age_max-age_diff-1,age_diff), sep="-"),
    paste(age_max-age_diff, "+", sep="")
    )
  ))
head(df)

In [ ]:
cohort1 <- df %>% filter(!is.na(date_event_heart) | !is.na(date_event_stroke) )  
cohort2 <- df %>% filter(((!is.na(surgery_elective_hip) | !is.na(surgery_elective_knee))) & !is.na(date_event))  
cohort3 <- df %>% filter(!is.na(date_event) , !is.na(acute_event_trauma))
head(cohort1)

In [ ]:
year_months <- data.frame(
  month=rep(1:12, year_to-year_from+1),
  year= rep(year_from:year_to,each=12))

In [ ]:
if(nrow(cohort1) != 0){
  stroke_per_month_year <- cohort1 %>% 
    mutate(month = month(date_event_stroke), year = year(date_event_stroke)) %>% dplyr::select(year,month,patient_id) %>% 
    aggregate(.,by=list(.$year, .$month), FUN=length) %>% 
    transmute("year" = Group.1, "month" = Group.2, total_stroke = patient_id) 
  
  stroke_per_month_year <- merge(year_months, stroke_per_month_year, by=c("year","month"),all.x = TRUE)
  stroke_per_month_year[is.na(stroke_per_month_year)] <- 0
  
  heart_per_month_year <- cohort1 %>% 
    mutate(month = month(date_event_heart), year = year(date_event_heart)) %>% dplyr::select(year,month,patient_id) %>% 
    aggregate(.,by=list(.$year, .$month), FUN=length) %>% 
    transmute("year" = Group.1, "month" = Group.2, total_heart_attack = patient_id) 
  
  heart_per_month_year <- merge(year_months, heart_per_month_year, by=c("year","month"),all.x = TRUE)
  heart_per_month_year[is.na(heart_per_month_year)] <- 0
  }else{
    stroke_per_month_year <- year_months
    stroke_per_month_year$total_stroke <- 0 
    
    heart_per_month_year <- year_months
    heart_per_month_year$total_heart_attack <- 0 
}

In [ ]:
if(nrow(cohort2) != 0){
  knee_per_month_year <- cohort2 %>% filter(!is.na(surgery_elective_knee)) %>% 
    mutate(month = month(date_event), year = year(date_event)) %>% dplyr::select(year,month,patient_id) %>% 
    aggregate(.,by=list(.$year, .$month), FUN=length) %>% 
    transmute("year" = Group.1, "month" = Group.2, total_knee_replacement = patient_id) 
  
  knee_per_month_year <- merge(year_months, knee_per_month_year, by=c("year","month"),all.x = TRUE)
  knee_per_month_year[is.na(knee_per_month_year)] <- 0
  
  hip_per_month_year <- cohort2 %>% filter(!is.na(surgery_elective_hip)) %>% 
    mutate(month = month(date_event), year = year(date_event)) %>% dplyr::select(year,month,patient_id) %>% 
    aggregate(.,by=list(.$year, .$month), FUN=length) %>% 
    transmute("year" = Group.1, "month" = Group.2, total_hip_replacement = patient_id) 
  
  hip_per_month_year <- merge(year_months, hip_per_month_year, by=c("year","month"),all.x = TRUE)
  hip_per_month_year[is.na(hip_per_month_year)] <- 0

}else{
  
  knee_per_month_year <- year_months
  knee_per_month_year$total_knee_replacement <- as.integer(0)
  hip_per_month_year <- year_months
  hip_per_month_year$total_hip_replacement <- as.integer(0)
}

In [ ]:
if(nrow(cohort3) != 0){
  acute_trauma_per_month_year <- cohort3 %>% 
  mutate(month = month(date_event), year = year(date_event)) %>% dplyr::select(year,month,patient_id) %>% 
  aggregate(.,by=list(.$year, .$month), FUN=length) %>% 
  transmute("year" = Group.1, "month" = Group.2, total_acute_trauma = patient_id) 
  
  acute_trauma_per_month_year <- merge(year_months, acute_trauma_per_month_year, by=c("year","month"),all.x = TRUE)
  acute_trauma_per_month_year[is.na(acute_trauma_per_month_year)] <- 0
}else{
  acute_trauma_per_month_year <- year_months
  acute_trauma_per_month_year$total_acute_trauma <- as.integer(0)
}

In [ ]:
summary_table <- merge(stroke_per_month_year, heart_per_month_year, by=c("year","month"),all.x = TRUE)
summary_table <- merge(summary_table, knee_per_month_year, by=c("year","month"),all.x = TRUE)
summary_table <- merge(summary_table, hip_per_month_year, by=c("year","month"),all.x = TRUE)
summary_table <- merge(summary_table, acute_trauma_per_month_year, by=c("year","month"),all.x = TRUE)
summary_table[is.na(summary_table)] <- 0

In [ ]:
dataset2 <- cohort1 %>% filter(!is.na(date_event_heart)) %>%
  mutate(month = month(date_event_heart), year = year(date_event_heart)) %>%  
  count(year,month,age_group,sex,socecon_lvl_cd) %>% rename(total_heart_attack = n )

dataset1 <-  cohort1 %>% filter(!is.na(date_event_stroke)) %>% 
  mutate(month = month(date_event_stroke), year = year(date_event_stroke)) %>%  
  count(year,month,age_group,sex,socecon_lvl_cd) %>% rename(total_stroke = n )

dataset3 <-  cohort2 %>% filter(!is.na(surgery_elective_knee)) %>% 
  mutate(month = month(date_event), year = year(date_event)) %>%  
  count(year,month,age_group,sex,socecon_lvl_cd) %>% rename(total_knee_replacement = n )

dataset4 <- cohort2 %>% filter(!is.na(surgery_elective_hip)) %>% 
  mutate(month = month(date_event), year = year(date_event)) %>%  
  count(year,month,age_group,sex,socecon_lvl_cd) %>% rename(total_hip_replacement = n )

dataset5 <-  cohort3 %>% 
  mutate(month = month(date_event), year = year(date_event)) %>%  
  count(year,month,age_group,sex,socecon_lvl_cd) %>% rename(total_acute_trauma = n )

In [ ]:
summary_table_aggregated <- merge(x=dataset1,y=dataset2,by=c("year","month","age_group","sex","socecon_lvl_cd"),all=TRUE)
summary_table_aggregated <- merge(x=summary_table_aggregated,y=dataset3,by=c("year","month","age_group","sex","socecon_lvl_cd"),all=TRUE)
summary_table_aggregated <- merge(x=summary_table_aggregated,y=dataset4,by=c("year","month","age_group","sex","socecon_lvl_cd"),all=TRUE)
summary_table_aggregated <- merge(x=summary_table_aggregated,y=dataset5,by=c("year","month","age_group","sex","socecon_lvl_cd"),all=TRUE)
summary_table_aggregated[is.na(summary_table_aggregated)] <- 0

In [ ]:
month_breaks <- c(1,2,3,4,5,6,7,8,9,10,11,12)
month_names <-
  c("Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec")


#############################################################################
#                             total_stroke                                    #
#############################################################################
p1<-ggplot(summary_table, aes(x=month, y=total_stroke, color = as.factor(year))) +
  geom_line(size = 1) +
  theme(panel.grid.major.x = element_line( color = "grey"),
        panel.grid.major.y = element_blank(), panel.border = element_blank(),
        panel.grid.minor = element_blank(), panel.background = element_blank(),
        axis.line = element_line(color = "black")) +
  theme(axis.text.x = element_text(angle = 90)) +
  geom_vline(xintercept = 3.76, color = "red", linetype = 2, size = 1) +
  labs(caption = paste0("n = ", sum(summary_table$total_stroke)),
       title = "Stroke", 
       color = "Year") +
  ylab(paste0("Absolute number of cases (N=", sum(summary_table$total_stroke),")"))  +
  expand_limits(y = 0) +
  scale_x_continuous(name = "Month", breaks = month_breaks[1:12], labels = month_names, expand = c(0,0)) +
  theme(plot.subtitle = element_text(hjust = 1), plot.caption = element_text(hjust = 1))

ggplotly(p1)

In [ ]:
highest_socecon_lvl <- max(levels(factor(df$socecon_lvl_cd)),na.rm = TRUE)
highest_socecon_lvl <- `highest_socecon_lvl`

dataAva <- nrow(cohort1 %>% filter(!is.na(date_event_stroke), date_event_stroke >= date_from, date_event_stroke <= date_to)) >0

if(!is.na(highest_socecon_lvl) && highest_socecon_lvl!=0 && dataAva){
  p1 <- 
  cohort1 %>% filter(!is.na(date_event_stroke), date_event_stroke >= date_from, date_event_stroke <= date_to) %>% 
  mutate(date_event = as.Date(paste(year(date_event_stroke),month(date_event_stroke),"01", sep = "-"))) %>% 
  group_by(date_event) %>%
  count(socecon_lvl_cd) %>%
  pivot_wider(id_cols = date_event, names_from = socecon_lvl_cd, values_from = n)

  p1 <- calculate_socecon_lvl_ratio(p1)  
  
  p1 <- p1 %>% 
  ggplot(aes(x=date_event, y=ratio)) +
  geom_line(size=1) +
  geom_smooth(method = "loess") +
  theme(panel.grid.major.x = element_line( color = "grey"),
        panel.grid.major.y = element_blank(), panel.border = element_blank(),
        panel.grid.minor = element_blank(), panel.background = element_blank(),
        axis.line = element_line(color = "black")) +
  theme(axis.text.x = element_text(angle = 90)) +
  coord_cartesian(ylim = c(0,2)) +
  labs(title = "Ratio of most to least deprived (stroke)",
       y= "Ratio most / least deprived socioeconomic level",
       x= "Time",
       color = "Year") +
  theme(plot.subtitle = element_text(hjust = 1), plot.caption = element_text(hjust = 1))
  
ggplotly(p1) %>%
  layout(title = list(text = paste0('Ratio of most to least deprived (stroke)',
                                    '<br>',
                                    '<sup>',
                                    '*Loess regression',
                                    '</sup>')))
}

In [ ]:
p1<-ggplot(summary_table, aes(x=month, y=total_heart_attack, color = as.factor(year))) +
  geom_line(size = 1) +
  theme(panel.grid.major.x = element_line( color = "grey"),
        panel.grid.major.y = element_blank(), panel.border = element_blank(),
        panel.grid.minor = element_blank(), panel.background = element_blank(),
        axis.line = element_line(color = "black")) +
  theme(axis.text.x = element_text(angle = 90)) +
  geom_vline(xintercept = 3.76, color = "red", linetype = 2, size = 1) +
  labs(subtitle = paste0("n = ", sum(summary_table$total_heart_attack)),
       title = "Heart attack", 
       color = "Year") +
  ylab(paste0("Absolute number of cases (N=", sum(summary_table$total_heart_attack),")"))  +
  expand_limits(y = 0) +
  scale_x_continuous(name = "Month", breaks = month_breaks[1:12], labels = month_names, expand = c(0,0)) +
  theme(plot.subtitle = element_text(hjust = 1), plot.caption = element_text(hjust = 1))

ggplotly(p1)

In [ ]:
dataAva <- nrow(cohort1 %>% filter(!is.na(date_event_heart), date_event_heart >= date_from, date_event_heart <= date_to)) >0

if(!is.na(highest_socecon_lvl) && highest_socecon_lvl!=0 && dataAva){
p1 <- 
  cohort1 %>% filter(!is.na(date_event_heart), date_event_heart >= date_from, date_event_heart <= date_to) %>% 
  mutate(date_event = as.Date(paste(year(date_event_heart),month(date_event_heart),"01", sep = "-"))) %>% 
  group_by(date_event) %>%
  count(socecon_lvl_cd) %>%
  pivot_wider(id_cols = date_event, names_from = socecon_lvl_cd, values_from = n)

  p1 <- calculate_socecon_lvl_ratio(p1)  
  
  p1 <- p1 %>% 
  ggplot(aes(x=date_event, y=ratio)) +
  geom_line(size=1) +
  geom_smooth(method = "loess") +
  theme(panel.grid.major.x = element_line( color = "grey"),
        panel.grid.major.y = element_blank(), panel.border = element_blank(),
        panel.grid.minor = element_blank(), panel.background = element_blank(),
        axis.line = element_line(color = "black")) +
  theme(axis.text.x = element_text(angle = 90)) +
  coord_cartesian(ylim = c(0,2)) +
  labs(title = "Ratio of most to least deprived (heart attack)",
       y= "Ratio most / least deprived socioeconomic level",
       x= "Time",
       color = "Year") +
  theme(plot.subtitle = element_text(hjust = 1), plot.caption = element_text(hjust = 1))
  
ggplotly(p1) %>%
  layout(title = list(text = paste0('Ratio of most to least deprived (heart attack)',
                                    '<br>',
                                    '<sup>',
                                    '*Loess regression',
                                    '</sup>')))
}

In [ ]:
p1<-ggplot(summary_table, aes(x=month, y=total_knee_replacement, color = as.factor(year))) +
  geom_line(size = 1) +
  theme(panel.grid.major.x = element_line( color = "grey"),
        panel.grid.major.y = element_blank(), panel.border = element_blank(),
        panel.grid.minor = element_blank(), panel.background = element_blank(),
        axis.line = element_line(color = "black")) +
  theme(axis.text.x = element_text(angle = 90)) +
  geom_vline(xintercept = 3.76, color = "red", linetype = 2, size = 1) +
  labs(subtitle = paste0("n = ", sum(summary_table$total_knee_replacement)),
       title = "Knee replacement", 
       color = "Year") +
  ylab(paste0("Absolute number of cases (N=", sum(summary_table$total_knee_replacement),")"))  +
  expand_limits(y = 0) +
  scale_x_continuous(name = "Month", breaks = month_breaks[1:12], labels = month_names, expand = c(0,0)) +
  theme(plot.subtitle = element_text(hjust = 1), plot.caption = element_text(hjust = 1))

ggplotly(p1)

In [ ]:
dataAva <- nrow(cohort2 %>% filter(!is.na(surgery_elective_knee), date_event >= date_from, date_event <= date_to)) >0

if(!is.na(highest_socecon_lvl) && highest_socecon_lvl!=0 && dataAva){
  p1 <- 
  cohort2 %>% filter(!is.na(surgery_elective_knee), date_event >= date_from, date_event <= date_to)  %>% 
  mutate(date_event = as.Date(paste(year(date_event),month(date_event),"01", sep = "-"))) %>% 
  group_by(date_event) %>%
  count(socecon_lvl_cd) %>%
  pivot_wider(id_cols = date_event, names_from = socecon_lvl_cd, values_from = n)

  p1 <- calculate_socecon_lvl_ratio(p1)  
  
  p1 <- p1 %>% 
  ggplot(aes(x=date_event, y=ratio)) +
  geom_line(size=1) +
  geom_smooth(method = "loess") +
  theme(panel.grid.major.x = element_line( color = "grey"),
        panel.grid.major.y = element_blank(), panel.border = element_blank(),
        panel.grid.minor = element_blank(), panel.background = element_blank(),
        axis.line = element_line(color = "black")) +
  theme(axis.text.x = element_text(angle = 90)) +
  coord_cartesian(ylim = c(0,2)) +
  labs(title = "Ratio of most to least deprived (knee replacement)",
       y= "Ratio most / least deprived socioeconomic level",
       x= "Time",
       color = "Year") +
  theme(plot.subtitle = element_text(hjust = 1), plot.caption = element_text(hjust = 1))
  
  ggplotly(p1) %>%
    layout(title = list(text = paste0('Ratio of most to least deprived (knee replacement)',
                                      '<br>',
                                      '<sup>',
                                      '*Loess regression',
                                      '</sup>')))
}

In [ ]:
p1<-ggplot(summary_table, aes(x=month, y=total_hip_replacement, color = as.factor(year))) +
  geom_line(size = 1) +
  theme(panel.grid.major.x = element_line( color = "grey"),
        panel.grid.major.y = element_blank(), panel.border = element_blank(),
        panel.grid.minor = element_blank(), panel.background = element_blank(),
        axis.line = element_line(color = "black")) +
  theme(axis.text.x = element_text(angle = 90)) +
  geom_vline(xintercept = 3.76, color = "red", linetype = 2, size = 1) +
  labs(subtitle = paste0("n = ", sum(summary_table$total_hip_replacement)),
       title = "Hip replacement", 
       color = "Year") +
  ylab(paste0("Absolute number of cases (N=", sum(summary_table$total_hip_replacement),")")) +
  expand_limits(y = 0)+
  scale_x_continuous(name = "Month", breaks = month_breaks[1:12], labels = month_names, expand = c(0,0)) +
  theme(plot.subtitle = element_text(hjust = 1), plot.caption = element_text(hjust = 1))

ggplotly(p1)

In [ ]:
dataAva <- nrow(cohort2 %>% filter(!is.na(surgery_elective_hip), date_event >= date_from, date_event <= date_to)) >0 

if(!is.na(highest_socecon_lvl) && highest_socecon_lvl!=0 && dataAva){
  p1 <- 
  cohort2 %>% filter(!is.na(surgery_elective_hip), date_event >= date_from, date_event <= date_to)  %>% 
  mutate(date_event = as.Date(paste(year(date_event),month(date_event),"01", sep = "-"))) %>% 
  group_by(date_event) %>%
  count(socecon_lvl_cd) %>%
  pivot_wider(id_cols = date_event, names_from = socecon_lvl_cd, values_from = n)

  p1 <- calculate_socecon_lvl_ratio(p1)  
  
  p1 <- p1 %>% 
  ggplot(aes(x=date_event, y=ratio)) +
  geom_line(size=1) +
  geom_smooth(method = "loess") +
  theme(panel.grid.major.x = element_line( color = "grey"),
        panel.grid.major.y = element_blank(), panel.border = element_blank(),
        panel.grid.minor = element_blank(), panel.background = element_blank(),
        axis.line = element_line(color = "black")) +
  theme(axis.text.x = element_text(angle = 90)) +
  coord_cartesian(ylim = c(0,2)) +
  labs(title = "Ratio of most to least deprived (hip replacement)",
       y= "Ratio most / least deprived socioeconomic level",
       x= "Time",
       color = "Year") +
  theme(plot.subtitle = element_text(hjust = 1), plot.caption = element_text(hjust = 1))
  
ggplotly(p1) %>%
  layout(title = list(text = paste0('Ratio of most to least deprived (hip replacement)',
                                    '<br>',
                                    '<sup>',
                                    '*Loess regression',
                                    '</sup>')))
}

In [ ]:
p1<-ggplot(summary_table, aes(x=month, y=total_acute_trauma, color = as.factor(year))) +
  geom_line(size = 1) +
  theme(panel.grid.major.x = element_line( color = "grey"),
        panel.grid.major.y = element_blank(), panel.border = element_blank(),
        panel.grid.minor = element_blank(), panel.background = element_blank(),
        axis.line = element_line(color = "black")) +
  theme(axis.text.x = element_text(angle = 90)) +
  geom_vline(xintercept = 3.76, color = "red", linetype = 2, size = 1) +
  labs(subtitle = paste0("n = ", sum(summary_table$total_acute_trauma)),
       title = "Acute severe trauma", 
       color = "Year") +
  ylab(paste0("Absolute number of cases (N=", sum(summary_table$total_acute_trauma),")")) +
  expand_limits(y = 0) +
  scale_x_continuous(name = "Month", breaks = month_breaks[1:12], labels = month_names, expand = c(0,0)) +
  theme(plot.subtitle = element_text(hjust = 1), plot.caption = element_text(hjust = 1))

ggplotly(p1)

In [ ]:
dataAva <- nrow(cohort3 %>% filter(date_event >= date_from, date_event <= date_to) ) >0 

if(!is.na(highest_socecon_lvl) && highest_socecon_lvl!=0 && dataAva ){
p1 <- 
  cohort3 %>% filter(date_event >= date_from, date_event <= date_to) %>% 
  mutate(date_event = as.Date(paste(year(date_event),month(date_event),"01", sep = "-"))) %>% 
  group_by(date_event) %>%
  count(socecon_lvl_cd) %>%
  pivot_wider(id_cols = date_event, names_from = socecon_lvl_cd, values_from = n)

  p1 <- calculate_socecon_lvl_ratio(p1)  
  
  p1 <- p1 %>% 
  ggplot(aes(x=date_event, y=ratio)) +
  geom_line(size=1) +
  geom_smooth(method = "loess") +
  theme(panel.grid.major.x = element_line( color = "grey"),
        panel.grid.major.y = element_blank(), panel.border = element_blank(),
        panel.grid.minor = element_blank(), panel.background = element_blank(),
        axis.line = element_line(color = "black")) +
  theme(axis.text.x = element_text(angle = 90)) +
  coord_cartesian(ylim = c(0,2)) +
  labs(title = "Ratio of most to least deprived (acute severe trauma)",
       y= "Ratio most / least deprived socioeconomic level",
       x= "Time",
       color = "Year") +
  theme(plot.subtitle = element_text(hjust = 1), plot.caption = element_text(hjust = 1))
  
ggplotly(p1)  %>%
  layout(title = list(text = paste0('Ratio of most to least deprived (acute severe trauma)',
                                    '<br>',
                                    '<sup>',
                                    '*Loess regression',
                                    '</sup>')))
}

### Summary table with total monthly cases by indicator ###

In [ ]:
datatable(summary_table, class = 'cell-border stripe')

In [ ]:
write.csv(summary_table_aggregated, file = "aggregated_outputs.csv", row.names = FALSE)